In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import pandas as pd
from prediction.model import TrajectoryTransformer, TrajectoryTrainer
from prediction.data import MaskedAISDataset

from torch.utils.data import DataLoader

from pathlib import Path

LOG_DIR = Path().resolve().parent.parent / "logs"

In [2]:
df = pd.read_pickle('ais_data.pkl')

In [3]:
MAX_SEQ_LEN = 100
dataset = MaskedAISDataset(df, MAX_SEQ_LEN, n_workers=1, normalize=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
device = "mps"

transformer = TrajectoryTransformer(
    d_model=128,
    nhead=4,
    num_encoder_layers=4,
    num_decoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len=MAX_SEQ_LEN
)

100%|██████████| 8585/8585 [00:04<00:00, 1998.00it/s]


In [4]:
trainer = TrajectoryTrainer(transformer, loader, loader, LOG_DIR, device=device)

In [5]:
trainer.train(max_epochs=10)

Epoch 0: 100%|██████████| 269/269 [00:22<00:00, 12.10it/s, loss=0.0793]


NotImplementedError: The operator 'aten::_nested_tensor_from_mask_left_aligned' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.